In [1]:
import os

os.chdir("..")
%load_ext autoreload

In [2]:
import matplotlib.pyplot as plt
import comet_ml
import numpy as np
import seaborn as sb
import pandas as pd
from src.transforms.motion_transform import Preprocess, FinalCrop
from src.network.SFCN import SFCNModel
from src.network.base_net import BaselineModel
import torch
from src.utils.mcdropout import evaluate_mcdropout

api = comet_ml.api.API(
    api_key="WmA69YL7Rj2AfKqwILBjhJM3k",
)

In [3]:
motion_volume = FinalCrop()(
    Preprocess()(
        {
            "data": r"C:\Users\Brico\Code\Master\mrart\data_test\ampscz\motion\sub-IR11959_ses-202305301_rec-norm_run-2_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz"
        }
    )
)
motion_volume["data"] = motion_volume["data"].unsqueeze(0)

clear_volume = FinalCrop()(
    Preprocess()(
        {
            "data": r"C:\Users\Brico\Code\Master\mrart\data_test\ampscz\clear\sub-MT03607_ses-202212291_rec-norm_run-1_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz"
        }
    )
)
clear_volume["data"] = clear_volume["data"].unsqueeze(0)

In [5]:
def enable_dropout(model):
    """Function to enable the dropout layers during test-time"""
    for m in model.modules():
        if m.__class__.__name__.startswith("Dropout"):
            m.train()


def get_mcdropout_list(model: torch.nn.Module, volume, n_samples=10):
    model.eval()
    enable_dropout(model)
    res = []
    for _ in range(n_samples):
        _, pred = model(volume)
        res.append(pred.sigmoid())
    return torch.Tensor(res)

In [6]:
ampscz_exps = api.get("mrart", "baseline-ampscz")
print(f"Found {len(ampscz_exps)} experiments")
for exp in ampscz_exps:
    print(exp.get_name())

Found 36 experiments
BASE
SERes
Conv5_FC3
SFCN
RES
ViT
BASE-3
BASE-1
BASE-2
BASE-5
BASE-4
RES-3
RES-1
RES-2
RES-5
RES-4
Conv5_FC3-1
Conv5_FC3-4
Conv5_FC3-3
Conv5_FC3-2
SFCN-3
SFCN-4
SFCN-1
SERes-1
SFCN-2
SERes-3
SERes-2
Conv5_FC3-5
SERes-5
SFCN-5
SERes-4
ViT-2
ViT-5
ViT-3
ViT-4
ViT-1


In [14]:
api.get("mrart", "estimate-motion", "pretraining-BASE-3").download_model(
    "BASE", output_path="BASE"
)
net = BaselineModel((1, 160, 192, 160), output_class=50)
net.load_state_dict(torch.load(f"BASE/model-data/comet-torch-model.pth"))

COMET INFO: Downloading experiment model 'BASE' ...
COMET INFO: Unzipping model to 'BASE' ...
COMET INFO: done!


76800


<All keys matched successfully>

In [ ]:
torch.jit.script(my_trained_model).save(my_zip_file_name)

In [7]:
preds = get_mcdropout_list(net, motion_volume["data"])
preds, preds.mean(), preds.std()

NameError: name 'net' is not defined

In [15]:
preds.round().int().bincount()

tensor([4, 6])

In [48]:
preds = get_mcdropout_list(net, clear_volume["data"])
preds, preds.mean(), preds.std()

(tensor([0.5388, 0.4671, 0.1978, 0.1168, 0.4177, 0.4036, 0.3181, 0.3963, 0.6650,
         0.2894]),
 tensor(0.3810),
 tensor(0.1601))

In [8]:
clear_volume["label"] = torch.IntTensor([0])
motion_volume["label"] = torch.IntTensor([1])

In [17]:
%autoreload 2
df = evaluate_mcdropout(net.cuda(), [clear_volume, motion_volume], None, n_samples=10)
df

100%|██████████| 10/10 [00:01<00:00,  8.44it/s]

metatensor([[ 4.,  5.,  7.,  4., 10.,  4.,  4.,  5.,  4.,  5.],
        [ 6.,  4.,  7.,  7., 10.,  7., 11.,  7.,  9.,  6.]]) [0, 1]
metatensor([5.2000, 7.4000]) metatensor([1.9322, 2.0656]) [[0 0 0 0 5 3 0 1 0 0 1 0]
 [0 0 0 0 1 0 2 4 0 1 1 1]]


,mean,std,labels,count
0,5.2,1.932184,0.0,"[0, 0, 0, 0, 5, 3, 0, 1, 0, 0, 1, 0]"
1,7.4,2.065591,1.0,"[0, 0, 0, 0, 1, 0, 2, 4, 0, 1, 1, 1]"
